In [1]:
import pandas as pd
import requests
import json
import cv2 as cv
import urllib.request
import numpy as np

In [2]:
df_anamoly = pd.read_csv(r"D:\Training\Azure\anomaly2.csv")
df_anamoly.rename(columns={"Unnamed: 0":"index"},inplace=True)
df_anamoly

,index,timestamp,False,True,Anomaly
0,48,2022-06-03T13:18:10Z,3.0,14.0,True
1,49,2022-06-03T13:18:10Z,3.0,14.0,True
2,50,2022-06-03T13:18:10Z,3.0,14.0,True
3,489,2022-06-03T13:20:37Z,5.0,10.0,True
4,490,2022-06-03T13:20:37Z,5.0,10.0,True
...,...,...,...,...,...
79,5731,2022-06-03T13:49:44Z,6.0,8.0,True
80,5732,2022-06-03T13:49:44Z,6.0,8.0,True
81,5868,2022-06-03T13:50:30Z,5.0,10.0,True
82,5869,2022-06-03T13:50:30Z,5.0,10.0,True


In [3]:
df_image = pd.read_csv(r"D:\Training\Azure\image.csv")
df_image.reset_index(inplace=True)
df_image['image_name'] = df_image['image_name'].str.replace("badsboys","badsboys2")
df_image.head()

,index,image_name,timestamp
0,0,https://roadanomalydetect.blob.core.windows.ne...,2022-06-03T13:14:25Z
1,1,https://roadanomalydetect.blob.core.windows.ne...,2022-06-03T13:14:25Z
2,2,https://roadanomalydetect.blob.core.windows.ne...,2022-06-03T13:14:25Z
3,3,https://roadanomalydetect.blob.core.windows.ne...,2022-06-03T13:14:25Z
4,4,https://roadanomalydetect.blob.core.windows.ne...,2022-06-03T13:14:25Z


In [4]:
df = pd.merge(df_anamoly,df_image,on='index',how='left')
df.dropna(inplace=True)

In [18]:
def predict(url):
    print('aaa')
    req = urllib.request.urlopen(url)
    arr = np.asarray(bytearray(req.read()), dtype=np.uint8)
    input = cv.imdecode(arr, -1) # 'Load it as it is'
    img_bgr = cv.cvtColor(input, cv.COLOR_RGB2BGR)
    crop_img = img_bgr[650:750, 750:1250]
    cv.imwrite(url.split('/')[-1], crop_img)

    

    ENDPOINT = "ENDPOINT"
    IMAGE = url.split('/')[-1]

    img_file = open(IMAGE, 'rb')

    response = requests.post(
            url = ENDPOINT,
            headers={
                "Content-Type": "application/octet-stream",
                "Prediction-Key": "Prediction-Key",
            },
            data=img_file,
        )

    return response.json()['predictions'][0]['tagName']

In [1]:
# df['image_class'] = df['image_name'].apply(predict)